# 学習したモデルを読み込んで推論のみ行う

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch
import transformers

transformers.logging.set_verbosity_error()

# モデルの読み込み
model = AutoPeftModelForCausalLM.from_pretrained(
    "./nagoyaben-lora-llama-3-8b-bnb-4bit",
    torch_dtype=torch.float16,
    device_map="auto",
)

# トークナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/llama-3-8b-bnb-4bit",
    trust_remote_code=True
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}
### Input:
{}
### Response:
{}"""

input_ids = tokenizer(
[
    alpaca_prompt.format(
        "名古屋弁で回答してください", # instruction
        "名古屋の名所を教えて？", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

# 推論の実行
output_ids = model.generate(
    **input_ids.to(model.device),
    max_new_tokens=200,
    do_sample=True,
    temperature=0.8,
)
output = tokenizer.decode(output_ids.tolist()[0])
print(output.split("### Response:\n")[1].split("<|end_of_text|>")[0])
